<a href="https://colab.research.google.com/github/Toshi-M56/PIDcontrol/blob/main/%E5%B7%A5%E5%AD%A6%E5%AE%9F%E9%A8%93PID%E5%88%B6%E5%BE%A1%E5%AE%9F%E9%A8%93%E3%82%B7%E3%83%9F%E3%83%A5%E3%83%AC%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# アクチュエーターのPID制御　実験シミュレーション

ここでは，電気電子工学実験2テーマ５　アクチュエータのPID制御の実験についてシミュレーションをしていきます．  

ちなみにこのホームページは[Colaboratory](https://colab.research.google.com/notebooks/intro.ipynb?hl=ja)と呼ばれるGoogleが提供するサービスで，  
Googleが提供するサーバを使って自由にPythonプログラムを記述・実行することができます．  
Webブラウザさえあればいいため，環境構築の手間がなく，  
PCのスペックが低くてもネット環境さえあれば本格的なPythonプログラムを作成して実行することができます．

## 使い方
まず初めに，このページの使い方を簡単に学んでいきましょう．
すぐ下に灰色の四角いボックスがあると思います．
ここに，Pythonのプログラムを書いていきます．

すぐ下のセル（灰色の枠）をクリックして「Shiftを押しながらEnter」とキーボードを打ってみてください．


In [ ]:
x = 1
print(x)

「1」と表示されたと思います．

これで
*   xという変数に1を代入
*   xの中身を画面に表示

という二つの命令を実行するプログラムを動かしたことになります．

このように，非常に簡単にPythonを実行できるのができるのがColaboratoryの便利なところです．

ちなみに，「#〜〜〜」でコメントアウトができます．

この部分はプログラムとして実行されないため，メモ書きとして利用できます．

以降のプログラムには，理解しやすいように時々メモ書きを残しておきますので，ご活用ください．

In [ ]:
# 2たす3を実行します
x = 2 + 3
print(x)

## シミュレーションの準備
プログラムが書けると，コンピューターに様々な処理を実行させることができます．しかし，膨大なプログラムの全ての処理を自分で書くのは大変です．そこで，ライブラリというすでに出来上がったプログラムを導入すると効率よくプログラムを作れます．

下のセルを実行してください．

「!pip 〜〜〜」という部分では，ライブラリをダウンロードし，

「import XXXXX as xx」という部分では，XXXXXというライブラリををxxという短い名前で使えるように導入しています．

細かいところは，今日は気にしないで大丈夫です．

In [ ]:
!pip install control
!pip install japanize_matplotlib
import numpy as np
from control import matlab
from matplotlib import pyplot as plt
import japanize_matplotlib
plt.style.use('bmh')

これで，

*   numpy (np) : 数値計算のためのライブラリ
*   matlab  : 制御に関するライブラリ
*   pyplot (plt) : グラフ表示に関するライブラリ
*   japanize_matplotlib (グラフに日本語を使うためのもの)

というライブラリを導入することができました．．


それでは早速matlab.tf()を使って以下のような伝達関数モデルを作りましょう．
$$ \frac{2}{s + 3} $$
以下を実行してください．

In [ ]:
# ()内の[2]とか[1, 3]は，伝達関数の分子と分母の係数を意味しています．
G = matlab.tf([2], [1, 3]) 
print(G)

このように，Gという変数に
$$\frac{2}{s+3}$$
という伝達関数を代入することができました．

そして，この関数のステップ応答を調べるには次のように実行します．

In [ ]:
# 0から5までを1000刻みに分割した数列をtに代入．
# つまり0sからから5sまでの時間．気になる人は人はprint(t)してみてください．
t = np.linspace(0, 10, 1000) 

# matlab.step()はステップ応答を求める関数．()の中には伝達関数Gと時間の数列tを入れます．
# すると，制御量(yout)と時間(T)が得られます．(Tはtと同じです)
yout, T = matlab.step(G, t) 

# plt.plot()はグラフをプロットします．
# ()の中にはx軸の値とy軸の値が入ります．ここではTとyoutですね．
plt.plot(T, yout)  
plt.show() #plt.show()でグラフを画面に表示します．

一連のコードをまとめると以下のようになります．
色々な伝達関数のステップ応答を試してみてもいいかもしれません．

In [ ]:
numerator = [6] # 分子
denominator = [1, 2, 6] # 分母
G = matlab.tf(numerator, denominator) 
print(G)
t = np.linspace(0, 10, 1000) 
yout, T = matlab.step(G, t) 
plt.plot(T, yout)  
plt.show() 

さて，このように伝達関数Gからステップ応答を求め，グラフを描写する処理を何度も書くのも面倒なので，

関数にまとめてしまいましょう．
そこで，次のようなplot_step_response()という関数を作りました．

()の中に伝達関数Gを入れるだけで，ステップ応答をグラフにしてくれます．
追加機能としてP制御とD制御の操作量をプロットできるようにしています．

興味がある人は中身を読んでみてください．

In [ ]:
def plot_step_response(G, control=None, time=5, n_step=1000):
    t = np.linspace(0, time, n_step) #時間の数列を作る time秒間を n_step分割で
    yout, T = matlab.step(G, t)  # ステップ応答を求める
    d_yout = (yout[1:] - yout[:-1])/(time/n_step) #位置の情報youtから 速度の情報情報d_youtを求める

    fig = plt.figure(figsize=(6,6)) # figureを生成
    ax = fig.add_subplot(211) # figureにsubplot(ax)を追加
    ax.plot(T, yout, c="blue", label="位置") # axにyout(位置)をプロット
    ax.legend(loc="upper right") # 凡例を表示する
    ax.axhline(1, color="black", linestyle="--") # 目標値を示す水平線をプロット
    ax.set_ylabel("位置", fontsize=20) # 位置の軸ラベルを表示する
    ax_v = ax.twinx() # 右側に速度グラフ用の軸を設けて subplot(ax_v)を重ねて追加
    ax_v.plot(T[1:], d_yout, c="r", label="速度") # d_yout(速度)をプロット
    ax_v.axhline(0, color="black", linestyle=":") # 速度0を示す水平線をプロット
    ax_v.set_ylabel("速度", fontsize=20) # 速度の軸ラベルを右の軸に表示
    ax_v.legend(loc="lower right") # 速度グラフの凡例を表示

    # ifの中は　P制御，PD制御の場合に操作量をプロットするためのもの．
    if control == "P" or control == "PD":
        ax2 = fig.add_subplot(212) # 新たなsubplot(ax2)を追加
        p_mvalue = (1-yout)*Kp # P制御の操作量を求める
        ax2.plot(T,p_mvalue, c="blueviolet", label="P制御") # P制御の操作量をプロット
        if control == "PD": # もしPD制御だったら
            d_mvalue = -d_yout*Kd # D制御の操作量を求める
            ax2.plot(T[:-1],d_mvalue, c="orange", label="D制御") # D制御の操作量をプロット
            ax2.plot(T[:-1],p_mvalue[:-1] + d_mvalue, c="green", label="PD制御") #PD制御の操作量をプロット
        ax2.axhline(0, color="black", linestyle="--") # 操作量0を示す水平線をプロット
        ax2.set_ylabel("操作量", fontsize=20) # 操作量の軸ラベルを表示
        ax2.legend() # 凡例を表示する
        ax2.set_xlabel("時間[s]", fontsize=20) # 横軸に時間の軸ラベルを表示
    plt.show() # 作成したグラフを表示

## P制御のシミュレーション
それではさっそくP制御のシミュレーションを実行していきましょう．

実験テキストのモデル式から導き出した制御対象の伝達関数は
$$P(s)=\frac{7}{s^2}$$
でした．
ここに，P制御
$$C(s)=K_p$$
をかけて閉ループ伝達関数を求めると
$$\frac{C(s)P(s)}{1+C(s)P(s)}=\frac{7K_p}{s^2 + 7K_p}$$
となるのでした．そこでひとまず
$$K_p=0.2$$
とし，この伝達関数のステップ応答を求めてみます．

In [ ]:
Kp = 0.2
G = matlab.tf([7*Kp], [1, 0, 7*Kp])
print("伝達関数G(s) ", G)

plot_step_response(G, control="P", time=10)



グラフが表示されたと思います．それぞれ

青：位置，　赤：速度，　黒：目標位置　紫：P制御の操作量

となっています．
グラフから気がつくことはありますか？

次の点についてグラフを観察しながら考えてみてください．

*   位置とP制御の操作量の関係
*   波形の周期ととKpの関係
*   位置と速度の関係
*   なぜ鋼球は止まらないのか，操作量の観点から考えてみる．

また，Kpの値を0.2, 0.4, 0.6, 0.8, 1.0と変えていくと何が起きますか？



##PD制御のシミュレーション
次に，PD制御のシミュレーションを実行していきましょう．

制御対象に，PD制御
$$C(s)=K_p + K_ds$$
をかけて閉ループ伝達関数を求めると
$$\frac{C(s)P(s)}{1+C(s)P(s)}=\frac{7K_ds + 7K_p}{s^2 + 7K_ds + 7K_p}$$
となるのでした．そこでひとまず
$$K_p=0.4, K_d=0.1$$
とし，この伝達関数のステップ応答を求めてみます．

In [ ]:
Kp = 0.4
Kd = 0.2
G = matlab.tf([7*Kd, 7*Kp], [1, 7*Kd, 7*Kp])
print("伝達関数G(s) ", G)

plot_step_response(G, control="PD", time=10)

グラフが表示されたと思います．それぞれ

青：位置，　赤：速度，　黒：目標位置　紫：P制御の操作量，　橙：D制御の操作量，　緑：PD制御の操作量

となっています．
グラフから気がつくことはありますか？

次の点についてグラフを観察しながら考えてみてください．

*   位置とP制御の操作量の関係
*   速度とD制御の操作量の関係
*   P制御，D制御，PD制御それぞれの関係
*   ステップ応答が振動しなくなる時（振動限界）の条件はなんですか？
*   $K_P=0.4$の時振動限界を迎えるを迎える$K_D$の値を値をPythonで求めてみましょう．

また，Kdの値を0.1, 0.2, 0.3, 0.4, 0.478, 0.5, 0.6と変えていくと何が起きますか？

In [ ]:
def function(t, Kp, Kd, show_D=True):
    D = 12.25*Kd**2 - 7*Kp

    if abs(D) < 1e-10:
        if show_D:print("D=" + str(D) + " :D = 0")
        term1 = np.ones(t.shape)
        term21 = -np.ones(t.shape)
        term22 = np.exp(-3.5*Kd*t) 
        term2 = term21 * term22
        term31 = -np.ones(t.shape) * (-3.5*Kd*t)
        term32 = np.exp(-3.5*Kd*t) 
        term3 =  term31 * term32
    elif D > 0:
        if show_D:print("D=" + str(D) + " :D > 0")
        beta = np.sqrt(12.25*Kd**2 - 7*Kp)
        # print(Kp, Kd, beta)
        p1 = -3.5*Kd + beta
        p2 = -3.5*Kd - beta
        # return 1 - (1 / (2*beta))* (p1*np.exp(p1*t) - p2*np.exp(p2*t))
        term1 = np.ones(t.shape)
        term21 = -(1 / (2*beta)) *term1
        term22 = p1*np.exp(p1*t) 
        term2 = term21 * term22
        term31 =  (1 / (2*beta)) *term1
        term32 =  p2*np.exp(p2*t)
        term3 =  term31 * term32
    elif D < 0:
        if show_D:print("D=" + str(D) + " :D < 0")
        omega = np.sqrt(-12.25*Kd**2 + 7*Kp)
        # return 1 - np.exp(-3.5*Kd*t) * (np.cos(omega*t) - (3.5*Kd/omega)*np.sin(omega*t))
        term1 = np.ones(t.shape)
        term21 = -np.cos(omega*t)
        term22 = np.exp(-3.5*Kd*t)
        term2 = term21 * term22
        term31 =  (3.5*Kd/omega)*np.sin(omega*t)
        term32 =  np.exp(-3.5*Kd*t) 
        term3 = term31 * term32
    return term1, term2, term3, term21, term22, term31, term32

In [ ]:
def plot_terms(t, term1, term2, term3, term21, term22, term31, term32):
    fig = plt.figure(figsize=(6,10))
    ax = fig.add_subplot(311)
    ax.plot(t,term1, c="violet",ls="--", label="第1項")
    ax.plot(t,term2, c="green", ls="--", label="第2項")
    ax.plot(t,term3, c="orange", ls="--", label="第3項")
    ax.plot(t,term1+term2+term3, c="black", label="位置")
    ax.set_ylim([-0.5, 1.7])
    ax.legend()
    ax2 = fig.add_subplot(312)
    ax2.plot(t,term21, ls="--", label="term21")
    ax2.plot(t,term22, ls="--", label="term22")
    ax2.plot(t,term2, c="green", label="第2項")
    ax2.legend()
    # ax2.set_ylim([-1.5, 1.5])
    ax3 = fig.add_subplot(313)
    ax3.plot(t,term31, ls="--", label="term31")
    ax3.plot(t,term32, ls="--", label="term32")
    ax3.plot(t,term3, c="orange", label="第3項")
    ax3.legend()
    # ax3.set_ylim([-1.5, 1.5])
    plt.show()

##PD制御について復讐
PD制御
$C(s)=K_P + K_Ds$
をかけた閉ループ伝達関数は
$$\frac{C(s)P(s)}{1+C(s)P(s)}=\frac{7K_Ds + 7K_P}{s^2 + 7K_Ds + 7K_P}$$
であり，その極は分母多項式から，
$$s=-3.5Kd \pm \sqrt{12.25K_D^2 - 7K_P}$$
となりました．これは判別式$D=12.25K_D^2-7K_P$によって3つに場合わけされます．

1.   $D<0$ 極が極が虚数成分を持つ
$$s = -3.5K_D \pm j \sqrt{7K_P - 12.25K_D^2}$$
2.   $D=0$　極が重根
$$s = -3.5K_D $$
3.   $D>0$　極が異なる二つの実数
$$s = -3.5K_D \pm j \sqrt{12.25K_D^2 - 7K_P}$$

##PD制御のステップ応答について復讐
そして，ステップ応答は
$$\frac{7K_Ds + 7K_P}{s(s^2 + 7K_Ds + 7K_P)}=\frac{1}{s}-\frac{1}{s^2 + 7K_Ds + 7K_P}$$
となり，ステップ応答の時間領域での式も判別式によって3つに別れました．
1.   $D<0$ 極が極が虚数成分を持つ
$$x(t) = 1 - e^{-3.5K_Dt} \big(\cos{\omega t} - \frac{3.5K_D}{\omega}\sin{\omega t} \big) =  1 - e^{-3.5K_Dt} \cos{\omega t} + e^{-3.5K_Dt} \frac{3.5K_D}{\omega}\sin{\omega t} $$
$$\big( \omega = \sqrt{7K_P - 12.25K_D^2} \big)$$
2.   $D=0$　極が重根
$$x(t)=1-e^{-3.5K_D t}(1-3.5K_D t)$$
3.   $D>0$　極が異なる二つの実数
$$1-\frac{1}{2 \beta}(p_1 e^{p_1 t} - p_2 e^{p_2 t})$$

$$= 1 + \frac{3.5K_D - \sqrt{12.25K_D^2 - 7K_P}}{2 \sqrt{12.25K_D^2 -7K_P}} e^{-3.5K_D + \sqrt{12.25K_D^2 - 7K_P}} - \frac{3.5K_D + \sqrt{12.25K_D^2 - 7K_P}}{2 \sqrt{12.25K_D^2 -7K_P}} e^{-(3.5K_D + \sqrt{12.25K_D^2 - 7K_P})}$$



In [ ]:
Kp = 0.4
Kd = 0.1
t = np.linspace(0, 10, 1000)
term1, term2, term3, term21, term22, term31, term32 = function(t, Kp, Kd)
plot_terms(t, term1, term2, term3, term21, term22, term31, term32)

In [ ]:
print(np.sqrt(7*Kp/12.25))
f = np.sqrt(7*Kp) / (2*np.pi)
print(1/f)
print(1)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc
from IPython.display import HTML


fig = plt.figure()

ims = []
for a in range(100):
    Kp = 0.4
    Kd = round(0.01*a, 4)
    t = np.linspace(0, 10, 1000)
    term1, term2, term3, term21, term22, term31, term32 = function(t, Kp, Kd, show_D=False)
    line, = plt.plot(t, term1, ls="--", c="violet")
    line2, = plt.plot(t, term2, ls="--", c="green")
    line3, = plt.plot(t, term3, ls="--", c="orange")
    line4, = plt.plot(t, term1+term2+term3, c="black")
    plt.ylim([-1.2,2.3])
    text = plt.text(2,2.2, "Kp = " + str(Kp) + "  Kd = " + str(Kd), fontsize=20)
    ims.append([line, line2, line3, line4, text])

ani = animation.ArtistAnimation(fig, ims)
rc('animation', html='jshtml')
plt.close()
ani
